In [2]:
import os
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import time

# 전체 코드 실행 시작 시간 기록
overall_start_time = time.time()
print(f"시작 시간: {time.ctime(overall_start_time)}")

# 데이터 디렉토리 설정
base_data_directory = './data/mfcc_data/'

# 이미지 크기 및 하이퍼파라미터 설정
img_height, img_width = 64, 64  # 이미지 크기를 더 작게 설정하여 메모리 사용량 줄이기
num_classes = 9  # 클래스 수 (예: babycry, boil, carhorn, door, emergencysiren, firesiren, gas, glass, stillsiren)

# 데이터셋 준비
X = []  # 이미지 데이터를 저장할 리스트
y = []  # 클래스 라벨을 저장할 리스트

# 데이터 디렉토리에서 이미지 파일을 불러와서 리스트에 추가
for filename in os.listdir(base_data_directory):
    if filename.endswith('.png'):  # PNG 파일만 처리
        # 클래스 번호 추출 (파일 이름에서 클래스 번호를 추출한다고 가정)
        class_number = int(filename.split('_')[0])
        # 이미지 파일 경로
        img_path = os.path.join(base_data_directory, filename)
        # 이미지 불러오기 및 전처리
        img = load_img(img_path, target_size=(img_height, img_width))  # 이미지 로드 및 크기 조정
        img_array = img_to_array(img) / 255.0  # 이미지를 배열로 변환 및 정규화 (0-1 범위)
        X.append(img_array.flatten())  # 이미지 배열을 평탄화하여 X 리스트에 추가
        y.append(class_number)  # 클래스 번호를 y 리스트에 추가

# 리스트를 numpy 배열로 변환
X = np.array(X)
y = np.array(y)

# 데이터셋을 학습 세트와 검증 세트로 나누기
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# PCA를 사용하여 데이터의 차원을 줄임
n_components = 100  # PCA 차원 수 설정
pca = PCA(n_components=n_components, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)

# SVM 모델 생성 및 학습
svm_model = SVC(kernel='linear', random_state=42)  # SVM 모델 생성 (선형 커널 사용)
svm_model.fit(X_train_pca, y_train)  # 학습 데이터로 모델 훈련

# 모델 평가
y_val_pred = svm_model.predict(X_val_pca)  # 검증 데이터로 예측

# 성능 보고서 및 혼동 행렬 출력
print("Classification Report:\n")
print(classification_report(y_val, y_val_pred, target_names=[str(i) for i in range(num_classes)]))  # 분류 보고서 출력
print("Confusion Matrix:\n")
print(confusion_matrix(y_val, y_val_pred))  # 혼동 행렬 출력

# 전체 코드 실행 종료 시간 기록
overall_end_time = time.time()
print(f"종료 시간: {time.ctime(overall_end_time)}")
print(f"소요 시간: {overall_end_time - overall_start_time:.2f}초")


시작 시간: Wed Jul 24 18:51:58 2024
Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       463
           1       1.00      0.99      1.00       452
           2       0.98      0.79      0.87       463
           3       0.99      1.00      0.99       437
           4       0.96      0.97      0.97       446
           5       0.82      0.99      0.90       454
           6       1.00      0.99      1.00       446
           7       1.00      1.00      1.00       450
           8       0.98      0.96      0.97       439

    accuracy                           0.97      4050
   macro avg       0.97      0.97      0.97      4050
weighted avg       0.97      0.97      0.97      4050

Confusion Matrix:

[[463   0   0   0   0   0   0   0   0]
 [  0 449   0   3   0   0   0   0   0]
 [  0   0 364   0   0  99   0   0   0]
 [  0   0   0 437   0   0   0   0   0]
 [  0   0   3   0 433   0   0   0  10]
 [  0   0   4   0   0 450